### Convolutional Neural Networks

#### Loading and Preparing the Data

In [134]:
import tensorflow.contrib.keras as keras
import pandas as pd
from tensorflow.contrib.keras.api.keras.callbacks import ReduceLROnPlateau
# Load in our data from CSV files
train_df = pd.read_csv("asl_data/sign_mnist_train.csv")
valid_df = pd.read_csv("asl_data/sign_mnist_valid.csv")

# Separate out our target values
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

# Separate out our image vectors
x_train = train_df.values
x_valid = valid_df.values

# Turn our scalar targets into binary categories
num_classes = 24
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

# Normalize our image data
x_train = x_train / 255
x_valid = x_valid / 255

### Reshaping Images for a CNN

In [135]:
x_train.shape, x_valid.shape

((27455, 784), (7172, 784))

Reshape our dataset so that they are in a 28x28 pixel format.   This will allow our convolutions to associate groups of pixels and detect important features.

In [136]:
x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

In [137]:
x_train.shape

(27455, 28, 28, 1)

In [138]:
x_valid.shape

(7172, 28, 28, 1)

In [139]:
x_train.shape, x_valid.shape

((27455, 28, 28, 1), (7172, 28, 28, 1))

#### Creating a Convolutional Model

In [141]:
from tensorflow.contrib.keras.api.keras.models import Sequential
from tensorflow.contrib.keras.api.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)
model = Sequential()
model.add(Conv2D(100, (3, 3), strides=1, padding="same", activation="relu", 
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.3))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(MaxPool2D((2, 2), strides=3, padding="same"))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(MaxPool2D((3, 3), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_97 (Conv2D)           (None, 28, 28, 100)       1000      
_________________________________________________________________
batch_normalization_65 (Batc (None, 28, 28, 100)       400       
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 28, 28, 75)        67575     
_________________________________________________________________
dropout_51 (Dropout)         (None, 28, 28, 75)        0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 28, 28, 50)        33800     
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 10, 10, 50)        0         
_________________________________________________________________
batch_normalization_66 (Batc (None, 10, 10, 50)        200       
__________

#### Summarizing the Model

In [142]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_97 (Conv2D)           (None, 28, 28, 100)       1000      
_________________________________________________________________
batch_normalization_65 (Batc (None, 28, 28, 100)       400       
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 28, 28, 75)        67575     
_________________________________________________________________
dropout_51 (Dropout)         (None, 28, 28, 75)        0         
_________________________________________________________________
conv2d_99 (Conv2D)           (None, 28, 28, 50)        33800     
_________________________________________________________________
max_pooling2d_78 (MaxPooling (None, 10, 10, 50)        0         
_________________________________________________________________
batch_normalization_66 (Batc (None, 10, 10, 50)        200       
__________

#### Compiling Model

In [143]:
model.compile(loss="categorical_crossentropy", metrics=["accuracy"],optimizer='adam')

#### Training

In [145]:
model.fit(x_train, y_train, epochs=25, verbose=1, validation_data=(x_valid, y_valid),batch_size = 128)

Train on 27455 samples, validate on 7172 samples
Epoch 1/25
27455/27455 [==============================] - 21s - loss: 0.0066 - acc: 0.9978 - val_loss: 0.0728 - val_acc: 0.9734
Epoch 2/25
27455/27455 [==============================] - 20s - loss: 0.0073 - acc: 0.9976 - val_loss: 0.0982 - val_acc: 0.9742
Epoch 3/25
27455/27455 [==============================] - 20s - loss: 0.0042 - acc: 0.9988 - val_loss: 0.0609 - val_acc: 0.9725
Epoch 4/25
27455/27455 [==============================] - 21s - loss: 0.0049 - acc: 0.9985 - val_loss: 0.0581 - val_acc: 0.9851
Epoch 5/25
27455/27455 [==============================] - 21s - loss: 0.0041 - acc: 0.9986 - val_loss: 0.0574 - val_acc: 0.9798
Epoch 6/25
27455/27455 [==============================] - 21s - loss: 0.0051 - acc: 0.9985 - val_loss: 0.1212 - val_acc: 0.9605
Epoch 7/25
27455/27455 [==============================] - 21s - loss: 0.0050 - acc: 0.9982 - val_loss: 0.1027 - val_acc: 0.9696
Epoch 8/25
27455/27455 [==============================]

#### Results

The training accuracy is very high, and the validation accuracy has improved as well. This is a great result, as all we had to do was swap in a new model.
But the validation accuracy jumping around. This is an indication that our model is still not generalizing perfectly. 